In [20]:
import re
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

import warnings
# ignore some deprecation warnings
warnings.filterwarnings('ignore')

# Load main data set

In [21]:
df = pd.read_csv('retrosynthesis-all', header=None)
df['source'] = df[0].apply(lambda x: x.split('>>')[0])
df['target'] = df[0].apply(lambda x: x.split('>>')[1])
df.drop(0, axis=1, inplace=True)
df.head()

,source,target
0,O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)...,CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc...
1,Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1,COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1
2,CC1(C)OB(c2cccc(Nc3nccc(C(F)(F)F)n3)c2)OC1(C)C,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.FC(F)(F)...
3,CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)O,CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)OCc1ccccc1
4,Fc1cc2c(NC3CCCCCC3)ncnc2cn1,Fc1cc2c(Cl)ncnc2cn1.NC1CCCCCC1


# SMILES Vocabulary: How is it generated?

The model's Vocabulary handles the transformation of SMILES strings into a sequence of tokens. Tokens are the pre-defined lowest and indivisible unit of string text. In Natural Language Processing (NLP), tokens are typically defined on the word or character level. The level of tokenization dictates *what* the model can output, e.g., if tokenization on the character level is used, then the model outputs individual characters.

For generative SMILES models, tokenization is performed on the character level where each token *loosely* maps to a unique atom type (brackets, "(" for example indicate branching and thus, do not map to an atom but rather gives connectivity information).


In [25]:
import sys
sys.path.append('src/')
from smiles_lstm.model.smiles_vocabulary import SMILESTokenizer, Vocabulary, create_vocabulary

tk = SMILESTokenizer()
vocab = Vocabulary()

#### Tokenization example

In [26]:
# smi_sample = df['source'].iloc[123]
smi_sample = 'CCBr'
print(tk.tokenize(smi_sample, with_begin_and_end=False))

['C', 'C', 'Br']


Much like in a natural language like english where one's vocabulary controls what sentences can be formulated, a molecular generative model's vocabulary controls what kind of atoms can be proposed - sentences in this context are molecules

In order for the token representations of SMILES sequences to be passed into a machine learning model, they must be transformed into a numerical representation. This is done in the Vocabulary class where each unique token is mapped to a numerical index. This is shown below:


In [27]:
# create a vocabulary using all SMILES in df
smiles_dataset = df['source'].unique().tolist()+ df['target'].unique().tolist()
smiles_dataset = np.unique(smiles_dataset).tolist()

vocabulary = create_vocabulary(smiles_list=smiles_dataset, tokenizer=tk)
print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'The unique tokens are: \n{vocabulary.tokens()}')

There are 86 unique tokens in the vocabulary.

The unique tokens are: 
['$', '^', ' ', '#', '(', ')', '-', '.', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', '[B-]', '[BH-]', '[BH3-]', '[Br-]', '[C-]', '[C@@H]', '[C@@]', '[C@H]', '[C@]', '[Cl+3]', '[Cl-]', '[Cu]', '[Fe]', '[I+]', '[K]', '[Li]', '[Mg+]', '[Mg]', '[N+]', '[N-]', '[N@+]', '[NH+]', '[NH-]', '[NH2+]', '[NH3+]', '[NH4+]', '[O-]', '[OH-]', '[P+]', '[PH2]', '[PH4]', '[PH]', '[Pd]', '[Pt]', '[S+]', '[S-]', '[S@@]', '[S@]', '[SH]', '[Se]', '[SiH2]', '[SiH]', '[Si]', '[SnH]', '[Sn]', '[Zn+]', '[Zn]', '[n+]', '[n-]', '[nH]', '[s+]', '[se]', '\\', 'c', 'n', 'o', 's']


In [28]:
print(smi_sample)
tokenized_smi_sample = tk.tokenize(smi_sample, with_begin_and_end=False)
print(tokenized_smi_sample)

vocabulary.encode(tokenized_smi_sample)

CCBr
['C', 'C', 'Br']


array([21., 21., 20.], dtype=float32)

# RNN section

This section describes *how* the numerical representation of tokens are transformed into an input vector known as the embedding that will act as the input to the RNN.

An Embedding Layer is essentially a look-up table. In the constructor above, `num_embeddings` refers to the Vocabulary size. 

`num_embeddings` denotes how many vectors to initialize. 

Since we have n unique tokens, we need _ different vectors: 1 for each unique token. This is why `num_embeddings` is _ in this example. `embedding_dim` denotes the dimension of the embedding vector. 5 is arbitrarily chosen here just for easy visualization.



In [29]:
# construct an "Embedding layer"
EMBEDDING_DIM = 5
NUM_EMBEDDING = len(vocabulary)

embedding_layer = nn.Embedding(num_embeddings=NUM_EMBEDDING,
                               embedding_dim=EMBEDDING_DIM)

# only 1 layer of LSTM cells is initialized here for the sake of illustration
# input_size = 5 because we previously defined the "embedding_dim" of the Embedding layer to be 5
# hidden_size = 5 is arbitrarily chosen for easy visualization
recurrent_layer = nn.LSTM(input_size=EMBEDDING_DIM,
                          hidden_size=5,
                          num_layers=1,
                          dropout=0,
                          batch_first=True)

In [30]:
# get the numerical indices of bromoethane
numerical_indices_smi_sample = torch.LongTensor([vocabulary.encode(tokenized_smi_sample).astype(int)])
print(f"Numerical indices of bromoethane:\n {numerical_indices_smi_sample}\n")

embedding = embedding_layer(numerical_indices_smi_sample)
print(f"Embedding:\n {embedding}")

Numerical indices of bromoethane:
 tensor([[21, 21, 20]])

Embedding:
 tensor([[[-1.4637, -0.8914,  0.2654, -0.5349, -0.8725],
         [-1.4637, -0.8914,  0.2654, -0.5349, -0.8725],
         [-0.8563, -0.6271,  0.2001, -0.6347, -1.1391]]],
       grad_fn=<EmbeddingBackward0>)


# Train / validation / test split

In [31]:
from sklearn.model_selection import train_test_split

print(df.shape)

# Splitting the data into train and combined val/test sets
train_data, val_test_data = train_test_split(df, test_size=0.2, random_state=42)

# Splitting the combined val/test set into separate val and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.2, random_state=42)

# Printing the sizes of the resulting splits
print("Train data size:", len(train_data))
print("Validation data size:", len(val_data))
print("Test data size:", len(test_data))

(45033, 2)
Train data size: 36026
Validation data size: 7205
Test data size: 1802


In [19]:
import sys
import pandas as pd
sys.path.append('src/')
import argparse
from pathlib import Path
from smiles_lstm.model.smiles_lstm import SmilesLSTM
from smiles_lstm.model.smiles_trainer import SmilesTrainer
from smiles_lstm.model.smiles_vocabulary import SMILESTokenizer, create_vocabulary
from smiles_lstm.utils import load
from smiles_lstm.utils.misc import suppress_warnings

zinc_path = Path("./scripts/data/zinc/")
train     = load.smiles(path=(zinc_path.joinpath("train.smi")))
test      = load.smiles(path=(zinc_path.joinpath("test.smi")))
valid     = load.smiles(path=(zinc_path.joinpath("valid.smi")))

dataset = train + test + valid

In [41]:
train     = train_data
test      = test_data
valid     = val_data

# create a vocabulary using all SMILES in df
dataset = df['source'].unique().tolist()+ df['target'].unique().tolist()
dataset = np.unique(dataset).tolist()

tokenizer = SMILESTokenizer()
vocab     = create_vocabulary(smiles_list=dataset,
                                    tokenizer=tokenizer,
                                    canonical=False)

print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'The unique tokens are: \n{vocabulary.tokens()}')

In [44]:
# trainer object expects data in a dictionary, so reorganizing it so
SMILES_dict = {"train" : train, "valid" : valid, "test"  : test}